In [18]:
import plotly.express as px
import pandas as pd
import tensorflow as tf
from drought_detection.data_handling import load_single_img
import numpy as np

In [4]:
STORAGE_LOCATION = "model" # local path

# load model (cache so it only loads once and saves time)
def load_model(STORAGE_LOCATION):
    return tf.saved_model.load(STORAGE_LOCATION)

model = load_model(STORAGE_LOCATION)

2022-07-21 20:20:17.836751: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [19]:
image, label = load_single_img(image_file='raw_data/train/part-r-00090', bands=['B4', 'B3', 'B2'], intensify=True)
image

array([[[105, 134, 137],
        [ 98, 129, 127],
        [109, 129, 137],
        ...,
        [129, 143, 147],
        [129, 138, 147],
        [113, 138, 147]],

       [[113, 138, 137],
        [ 98, 120, 132],
        [ 98, 120, 132],
        ...,
        [129, 138, 147],
        [121, 138, 147],
        [113, 134, 147]],

       [[102, 120, 132],
        [ 90, 116, 122],
        [ 82, 102, 127],
        ...,
        [125, 138, 142],
        [156, 161, 153],
        [133, 147, 147]],

       ...,

       [[117, 134, 142],
        [117, 134, 142],
        [113, 134, 142],
        ...,
        [184, 174, 188],
        [176, 170, 183],
        [160, 161, 173]],

       [[109, 129, 142],
        [105, 129, 142],
        [105, 129, 142],
        ...,
        [141, 152, 163],
        [133, 143, 158],
        [121, 134, 158]],

       [[109, 129, 142],
        [105, 129, 142],
        [102, 129, 142],
        ...,
        [ 66, 107, 122],
        [105, 129, 142],
        [149, 161, 168]]

In [20]:
image = np.expand_dims(a=np.array(image), axis=0)

In [43]:
def predict(image):
    #predict from web:
    with tf.compat.v1.Session(graph=tf.Graph()) as sess:
        tf.compat.v1.saved_model.loader.load(sess, [tf.compat.v1.saved_model.tag_constants.SERVING], STORAGE_LOCATION)
        graph = tf.compat.v1.get_default_graph()
        prediction = sess.run('StatefulPartitionedCall:0',
                                feed_dict={'serving_default_keras_layer_input:0': image})
    return prediction

prediction = predict(image)
prediction

INFO:tensorflow:Restoring parameters from model/variables/variables


array([[0.58093226, 0.00416328, 0.41072002, 0.00418442]], dtype=float32)

In [44]:
def get_dataframe_data(prediction):
            df = pd.DataFrame(prediction.transpose())
            df[0] = (df[0] * 100).round(decimals = 2)
            df.reset_index(inplace=True)
            df = df.rename(columns = {'index':'Classification', 0:f"% Likelihood"})
            return df
# get dataframe
df = get_dataframe_data(prediction)
# show dataframe
df['Number of cows'] = ['0 cows', '1 cow', '2 cows', '3+ cows']


In [45]:
df.iloc[:,1]

0    58.09
1     0.42
2    41.07
3     0.42
Name: % Likelihood, dtype: float32

In [86]:

def make_fig(df, x, y):
    fig = px.scatter(df, x, y,
                        size=f"% Likelihood", color='Number of cows',
                        color_discrete_sequence=px.colors.qualitative.Vivid, 
                        hover_name=f"% Likelihood", size_max=100)
    fig.update_coloraxes(showscale=False)
    full_fig = fig.update_layout(
        plot_bgcolor='rgba(0, 0, 0, 0)',
        title="Regional Drought Likelihood",
        xaxis_title="# of Cows",
        yaxis_title="% Confidence",
        font=dict(
            family="Open Sans, verdana, arial, sans-serif",
            size=25,
            color="#7f7f7f"))
    return full_fig

In [87]:
full_fig